In [1]:
import pandas as pd

In [2]:
def_df = pd.read_csv("/Users/alyssaliu/Desktop/StatsM148/Event Definitions.csv")
df = pd.read_csv("/Users/alyssaliu/Desktop/StatsM148/export.csv")

In [3]:
def_df

,event_name,journey_id,event_definition_id,milestone_number,stage
0,application_phone_approved,1,15,1.0,Apply for Credit
1,application_phone_declined,1,16,NaN,Apply for Credit
2,application_phone_pending,1,17,NaN,Apply for Credit
3,application_web_approved,1,12,1.0,Apply for Credit
4,application_web_declined,1,13,NaN,Apply for Credit
5,application_web_pending,1,14,NaN,Apply for Credit
6,application_web_submit,1,3,NaN,Apply for Credit
7,application_web_view,1,19,NaN,Apply for Credit
8,account_activitation,1,29,3.0,Credit Account
9,account_fraud_review,1,37,NaN,Credit Account


In [4]:
print(df.shape)
df.head()

(64911906, 6)


,customer_id,account_id,ed_id,event_name,event_timestamp,journey_steps_until_end
0,-784961211,1773350293,12,application_web_approved,2023-03-22T08:45:22.000Z,1
1,-784961211,1773350293,19,application_web_view,2023-03-22T13:32:10.000Z,2
2,-784961211,1773350293,19,application_web_view,2023-03-22T13:32:10.000Z,3
3,-784961211,1773350293,19,application_web_view,2023-03-22T13:32:10.000Z,4
4,-784961211,1773350293,19,application_web_view,2023-03-22T13:32:10.000Z,5


In [5]:
df['event_timestamp'] = pd.to_datetime(df['event_timestamp']) # convert to pd datetime

## First deal with removing duplicate events

Ben said these could be ignored

In [6]:
# remove duplicate rows - don't factor in journey_steps_until_end because this counts up even for duplicate rows
df_dropped = df.drop(['journey_steps_until_end'], axis = 1)
df_dropped = df_dropped.drop_duplicates()
# df_dropped = df.drop(['journey_steps_until_end'], axis = 1)
df_dropped.head(10)

,customer_id,account_id,ed_id,event_name,event_timestamp
0,-784961211,1773350293,12,application_web_approved,2023-03-22 08:45:22+00:00
1,-784961211,1773350293,19,application_web_view,2023-03-22 13:32:10+00:00
14,-784961211,1773350293,3,application_web_submit,2023-03-22 13:32:10+00:00
15,-784961211,1773350293,2,campaign_click,2023-03-22 14:45:22+00:00
16,-784961211,1773350293,19,application_web_view,2023-07-27 14:57:56+00:00
21,-784961211,1773350293,19,application_web_view,2023-08-29 16:01:06+00:00
24,15849251,383997507,4,browse_products,2021-11-04 14:11:15+00:00
25,15849251,383997507,4,browse_products,2021-11-04 14:11:29+00:00
26,15849251,383997507,4,browse_products,2021-11-04 14:12:10+00:00
27,15849251,383997507,4,browse_products,2021-11-04 14:12:21+00:00


In [7]:
print(df_dropped.shape)
df.shape[0] - df_dropped.shape[0] # number of duplicate rows removed ~8 million

(56853035, 5)


8058871

In [8]:
df_dropped['journey_steps_until_end'] = df_dropped.groupby(['customer_id', 'account_id']).cumcount() + 1 # add journey_steps_until_end back in

In [9]:
df_dropped.reset_index(drop=True, inplace=True)
df_dropped.head(10)

,customer_id,account_id,ed_id,event_name,event_timestamp,journey_steps_until_end
0,-784961211,1773350293,12,application_web_approved,2023-03-22 08:45:22+00:00,1
1,-784961211,1773350293,19,application_web_view,2023-03-22 13:32:10+00:00,2
2,-784961211,1773350293,3,application_web_submit,2023-03-22 13:32:10+00:00,3
3,-784961211,1773350293,2,campaign_click,2023-03-22 14:45:22+00:00,4
4,-784961211,1773350293,19,application_web_view,2023-07-27 14:57:56+00:00,5
5,-784961211,1773350293,19,application_web_view,2023-08-29 16:01:06+00:00,6
6,15849251,383997507,4,browse_products,2021-11-04 14:11:15+00:00,1
7,15849251,383997507,4,browse_products,2021-11-04 14:11:29+00:00,2
8,15849251,383997507,4,browse_products,2021-11-04 14:12:10+00:00,3
9,15849251,383997507,4,browse_products,2021-11-04 14:12:21+00:00,4


In [10]:
df_dropped.dtypes

customer_id                              int64
account_id                               int64
ed_id                                    int64
event_name                              object
event_timestamp            datetime64[ns, UTC]
journey_steps_until_end                  int64
dtype: object

## Look into removing incomplete customers

Determining whether a customer is unsuccessful (quit) or incomplete (the data was just cut off, so we don't know if they ended up having a successful journey or not) 

according to Ben: it is a bit tricky to know if it’s a ‘quit’ or an ‘incomplete’.  What we typically look for is 60+ days since an approval, after that point we have VERY few people order if they haven’t yet.  Here’s our activation curve as of late as an example:

![Image Description](cum_activation.png)

I will create a dataframe for customers who have had 70+ days since activation - to weed out incomplete customers

In [17]:
# get count of account_activations for each customer/account pair
multiple_activations = df_dropped.groupby(['customer_id', 'account_id'])['event_name'].apply(lambda x: x.str.count('account_activitation').sum())

In [16]:
multiple_activations.value_counts()

event_name
0    1319326
1     414268
2       2013
3        160
Name: count, dtype: int64

In [22]:
multiple_activations

customer_id  account_id 
-2147483541   2059949086    0
-2147481037  -1245770865    0
-2147474335  -1929965268    1
-2147474305   1098145658    0
-2147472167   889210381     1
                           ..
 2147480051  -148210516     0
 2147480182  -1286764612    0
 2147480920   496493954     0
 2147482120  -488077144     0
 2147483471   1556733004    0
Name: event_name, Length: 1735767, dtype: int64

In [43]:
multiple_activations = multiple_activations.reset_index()
filt = ~df_dropped.set_index(['customer_id', 'account_id']).index.isin(multiple_activations.set_index(['customer_id', 'account_id']).index)
df_filtered = df_dropped[filt].reset_index(drop=True)

In [45]:
df_filtered

,customer_id,account_id,ed_id,event_name,event_timestamp,journey_steps_until_end


In [44]:
# check that all customer/account combos only have 1 activation
df_filtered.groupby(['customer_id', 'account_id'])['event_name'].apply(lambda x: x.str.count('account_activitation').sum()).value_counts()

Series([], Name: count, dtype: int64)

#### It seems that multiple account activations just indicates multiple journeys under the same customer_id and account_id which is interesting

Because such a small portion of data was removed by removing customer with multiple activations, we will proceed with df_filtered

In [15]:
df_dropped[['customer_id', 'account_id']].drop_duplicates().shape # 1735767 unique combinations of customer/account id

(1735767, 2)

In [16]:
print(2173 / 1735767) # 0.001% of customer/account combinations would be removed if we removed those with multiple account activations
238751 / df_dropped.shape[0] # 0.4% of dataset would be dropped if we removed customers with multiple activations

0.0012518961358292905


0.004199441595334356

Because such a small portion of the dataset has multiple activations, we will drop these rows for simplicity and for the sake of keeping the data uniform.

We will proceed with attempting to remove "incomplete" customers in the data

In [17]:
# function to remove all rows with customer/account combinations that have multiple activations
df_dropped_filtered = df_dropped[~df_dropped[['customer_id', 'account_id']].isin(multiple_activations.values.flatten()).all(axis = 1)]
df_dropped_filtered.head()

,customer_id,account_id,ed_id,event_name,event_timestamp,journey_steps_until_end
0,-784961211,1773350293,12,application_web_approved,2023-03-22 08:45:22+00:00,1
1,-784961211,1773350293,19,application_web_view,2023-03-22 13:32:10+00:00,2
2,-784961211,1773350293,3,application_web_submit,2023-03-22 13:32:10+00:00,3
3,-784961211,1773350293,2,campaign_click,2023-03-22 14:45:22+00:00,4
4,-784961211,1773350293,19,application_web_view,2023-07-27 14:57:56+00:00,5


In [18]:
print(df_dropped.shape)
df_dropped_filtered.shape

(56853035, 6)


(56614284, 6)

### Now check account_activation counts - should all be either 1 or 0

In [19]:
df_dropped_filtered.groupby(['customer_id', 'account_id']).size().reset_index(name='count')

,customer_id,account_id,count
0,-2147483541,2059949086,29
1,-2147481037,-1245770865,7
2,-2147474335,-1929965268,25
3,-2147474305,1098145658,32
4,-2147472167,889210381,39
...,...,...,...
1733589,2147480051,-148210516,14
1733590,2147480182,-1286764612,9
1733591,2147480920,496493954,110
1733592,2147482120,-488077144,3


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x1130d0f10>>
Traceback (most recent call last):
  File "/Users/alyssaliu/Library/Python/3.11/lib/python/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [ ]:
activation_counts

In [ ]:
activations = df_dropped[df_dropped['event_name'] == 'account_activitation']
activation_dates = activations.groupby(['customer_id', 'account_id'])['event_timestamp'].min().reset_index()
activation_dates.rename(columns={'event_timestamp': 'activation_date'}, inplace=True)
activation_dates

In [60]:
cutoff_date = df_dropped['event_timestamp'].max()
activation_dates['days_since_activation'] = (cutoff_date - activation_dates['activation_date']).dt.days
# we will consider these incomplete customers and remove them
filtered_customers = activation_dates[activation_dates['days_since_activation'] >= 70] # remove customers who activated less than 70 days ago

# now merge back
df_filtered = df_dropped.merge(filtered_customers[['customer_id', 'account_id']], on=['customer_id', 'account_id'], how='inner')

In [61]:
df_filtered.head()

,customer_id,account_id,ed_id,event_name,event_timestamp,journey_steps_until_end
0,-1257367741,1076891306,19,application_web_view,2023-01-04 17:41:54+00:00,1
1,-1257367741,1076891306,19,application_web_view,2023-01-04 17:41:56+00:00,2
2,-1257367741,1076891306,19,application_web_view,2023-01-04 17:41:57+00:00,3
3,-1257367741,1076891306,19,application_web_view,2023-01-04 17:43:00+00:00,4
4,-1257367741,1076891306,19,application_web_view,2023-01-04 17:43:08+00:00,5


In [64]:
print(df_dropped.shape)
print(df_filtered.shape)
print(df_dropped.shape[0] - df_filtered.shape[0]) # removed

(56853035, 6)
(21533943, 6)
35319092


## Now onto data cleaning - turning data into wide format

In [28]:
# Group by 'customer_id' and aggregate the other columns into lists
df_grouped = df_dropped.groupby('customer_id').agg({
    'account_id': list,
    'ed_id': list,
    'event_name': list,
    'event_timestamp': list,
    'journey_steps_until_end': list,
}).reset_index()

In [29]:
df_grouped.head(2)

,customer_id,account_id,ed_id,event_name,event_timestamp,journey_steps_until_end
0,-2147483541,"[2059949086, 2059949086, 2059949086, 205994908...","[12, 5, 4, 11, 5, 6, 1, 4, 11, 4, 11, 5, 6, 5,...","[application_web_approved, view_cart, browse_p...","[2021-10-05 08:11:26+00:00, 2021-10-05 08:17:1...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,-2147481037,"[-1245770865, -1245770865, -1245770865, -12457...","[12, 1, 2, 5, 11, 4, 6]","[application_web_approved, promotion_created, ...","[2023-08-26 12:08:34+00:00, 2023-08-26 17:18:4...","[1, 2, 3, 4, 5, 6, 7]"


In [30]:
df_grouped.shape

(1665688, 6)

In [31]:
df_grouped['account_id'] = df_grouped['account_id'].apply(lambda x: list(set(x)))

In [32]:
df_grouped.head()

,customer_id,account_id,ed_id,event_name,event_timestamp,journey_steps_until_end
0,-2147483541,[2059949086],"[12, 5, 4, 11, 5, 6, 1, 4, 11, 4, 11, 5, 6, 5,...","[application_web_approved, view_cart, browse_p...","[2021-10-05 08:11:26+00:00, 2021-10-05 08:17:1...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,-2147481037,[-1245770865],"[12, 1, 2, 5, 11, 4, 6]","[application_web_approved, promotion_created, ...","[2023-08-26 12:08:34+00:00, 2023-08-26 17:18:4...","[1, 2, 3, 4, 5, 6, 7]"
2,-2147474335,[-1929965268],"[2, 12, 19, 3, 4, 4, 19, 4, 1, 11, 4, 5, 11, 4...","[campaign_click, application_web_approved, app...","[2023-04-14 01:42:50+00:00, 2023-04-14 19:42:5...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,-2147474305,[1098145658],"[6, 19, 19, 19, 19, 19, 19, 19, 1, 4, 19, 19, ...","[begin_checkout, application_web_view, applica...","[2021-11-28 22:27:11+00:00, 2021-11-28 22:28:0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,-2147472167,[889210381],"[12, 19, 3, 4, 4, 1, 1, 11, 5, 4, 11, 5, 4, 6,...","[application_web_approved, application_web_vie...","[2023-02-19 08:23:54+00:00, 2023-02-19 14:21:5...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [33]:
df_grouped.to_pickle("wide_format_data_no_duplicates.pkl")

In [12]:
# df_grouped.to_csv('wide_format_df.csv')
# this doesn't preserve lists in the columns, turns them into strings

In [35]:
df['event_name'].value_counts()

event_name
browse_products                           23099680
application_web_view                      10249155
view_cart                                  7128768
promotion_created                          6887889
add_to_cart                                4441843
begin_checkout                             2885054
catalog_(mail)                             2242333
application_web_approved                   1695968
campaignemail_clicked                      1637935
application_web_submit                     1420199
campaign_click                             1042214
place_downpayment                           428050
account_activitation                        418774
place_order_web                             345773
account_downpaymentcleared                  333704
order_shipped                               322949
pre-application_(3rd_party_affiliates)      151175
place_order_phone                            63656
application_web_declined                     43139
application_phone_ap

In [38]:
account_act_count = df_grouped['event_name'].apply(lambda x: x.count('account_activitation'))
account_act_count


0          0
1          0
2          1
3          0
4          1
          ..
1665683    0
1665684    0
1665685    0
1665686    0
1665687    0
Name: event_name, Length: 1665688, dtype: int64

In [45]:
mult_activation = df_grouped[df_grouped['event_name'].apply(lambda x: x.count('account_activitation') > 1)]


In [41]:
account_act_count.value_counts()

event_name
0    1254817
1     403355
2       7148
3        351
4         15
5          2
Name: count, dtype: int64

count
1    414268
2      2013
3       160
Name: count, dtype: int64